In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res


In [3]:
### 目标函数：

bounds = [(-1, 1), (-1, 1)]
x_opt = [-0.707036070037170616, 0.500000004333606807]
y_opt = 0.7499


def G11(X):
    y = X[0] ** 2 + (X[1] - 1) ** 2

    h = X[1] - X[0] ** 2 - 0.0001

    return y, h

In [4]:
### 适应度函数
def fitness1_num(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1220.174
    fmin = 0.008509619375000003
    count = np.sum(np.array(problem1(X))[1:5]>0)
    
    return -(problem1(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG11_vio(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 5
    fmin = 0
    vmax = 1
    e = 0.1
    n = 1
    
    res = G11(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:2]>0)
    
    if count > 0:
        for i in range(1,len(res)):
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
    return fitness   

In [5]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-100000000.0)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-100000000.0)    # 全局最佳适应值
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG11_vio(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
            w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG11_vio(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            print(self.g_fit)  
        return self.gbest, self.g_fit

In [6]:
PSO = PSO_fit(pN=400, dim=2, max_iter=100, bounds=bounds)

In [7]:
PSO.init_Population()

In [8]:
gbest,gfit = PSO.iterator()

[-0.98457087 -1.00325613 -1.94400935 -0.91437627 -0.91849001 -1.0891236
 -0.91849001 -0.87430761 -0.87428122 -1.54363845 -0.94629659 -0.82512551
 -0.86065199 -1.16345984 -1.04952589 -1.         -0.90403691 -0.79318196
 -1.         -1.16345984 -0.90114621 -1.0891236  -0.89692538 -0.99066693
 -0.86065199 -1.54988914 -1.05754331 -1.01434538 -1.27382732 -1.08300546
 -2.01225329 -1.         -0.76102432 -0.79318196 -1.37375604 -1.54988914
 -0.79318196 -1.56098895 -1.         -0.89692538 -0.91486216 -0.86065199
 -0.79318196 -1.         -0.82512551 -0.79318196 -1.         -1.08359279
 -0.91330999 -1.         -0.83641876 -1.09230108 -1.         -1.89592144
 -0.95198909 -1.00855972 -0.9829555  -1.10206466 -0.86970196 -1.01769731
 -1.         -0.89692538 -1.01434538 -0.87921349 -0.87428122 -0.78143819
 -1.10206466 -0.78364984 -0.90479728 -1.11366861 -0.91330999 -1.27382732
 -1.         -0.9829555  -0.89692538 -0.891636   -0.891636   -0.95888498
 -1.01388673 -0.78143819 -1.29370183 -0.95198909 -0.

[-0.7766418  -0.79318196 -0.7951048  -0.77551464 -0.75809655 -0.83258464
 -0.77327585 -0.7951048  -0.78143819 -0.76153104 -0.77551464 -0.77468624
 -0.7843214  -0.76279089 -0.76102432 -0.77929919 -0.76974939 -0.75809655
 -0.77339027 -0.76764693 -0.76504491 -0.76167399 -0.76504491 -0.77014112
 -0.76102432 -0.76751201 -0.76974939 -0.76504491 -0.77559319 -0.79329592
 -0.77468624 -0.76102432 -0.76102432 -0.77929919 -0.77339027 -0.76167399
 -0.78828165 -0.78949973 -0.77014112 -0.7766418  -0.76167399 -0.76167399
 -0.78449712 -0.78143819 -0.77805821 -0.77479729 -0.7843214  -0.78143819
 -0.79040837 -0.77929919 -0.77559319 -0.78364984 -0.75809655 -0.78449712
 -0.76102432 -0.76764693 -0.76572545 -0.76974939 -0.77327585 -0.76102432
 -0.77559319 -0.76504491 -0.77625269 -0.77625269 -0.79318196 -0.76504491
 -0.7843214  -0.76102432 -0.78576705 -0.77551464 -0.78574493 -0.76167399
 -0.77551464 -0.785496   -0.75809655 -0.79329592 -0.78243351 -0.76102432
 -0.78781184 -0.77479729 -0.76974939 -0.76389907 -0

[-0.76348016 -0.75889434 -0.75574824 -0.75665651 -0.75809655 -0.76389907
 -0.75748088 -0.76167399 -0.75972465 -0.75748088 -0.76208023 -0.76153104
 -0.75809655 -0.75556617 -0.75953906 -0.76208023 -0.75972465 -0.75556617
 -0.76872266 -0.75748088 -0.76344829 -0.75948903 -0.75556617 -0.75678846
 -0.76102432 -0.75748088 -0.75708672 -0.75889434 -0.75708672 -0.76208023
 -0.75574824 -0.75548618 -0.75574824 -0.75788317 -0.75678846 -0.75708672
 -0.75972465 -0.75972465 -0.76102432 -0.75214945 -0.76167399 -0.75622952
 -0.76572545 -0.75889434 -0.77200378 -0.75678846 -0.76110375 -0.75113521
 -0.75622952 -0.75529046 -0.75556617 -0.75719619 -0.75665651 -0.75788317
 -0.75113521 -0.75809655 -0.76208023 -0.76348016 -0.76330094 -0.75972465
 -0.76102432 -0.75788317 -0.75113521 -0.75889434 -0.75678846 -0.75556617
 -0.76817541 -0.76102432 -0.75113521 -0.75665651 -0.75556617 -0.75788317
 -0.76330094 -0.75622952 -0.75556617 -0.75748088 -0.75948903 -0.76102432
 -0.75665651 -0.75622952 -0.75574824 -0.75622952 -0

 -0.75214945 -0.75113521 -0.75298478 -0.75113521]
[-0.75214945 -0.75376517 -0.75214945 -0.75288419 -0.75214945 -0.75604227
 -0.75183046 -0.75403999 -0.75348109 -0.75343834 -0.75214887 -0.75438723
 -0.75529046 -0.7532395  -0.75113521 -0.75578199 -0.75113521 -0.75023659
 -0.75023659 -0.75298478 -0.75214887 -0.75113521 -0.75376517 -0.75264804
 -0.75214945 -0.75340329 -0.75362944 -0.7532395  -0.75125453 -0.75288419
 -0.75113521 -0.7548147  -0.75342605 -0.75288419 -0.7532395  -0.7562651
 -0.75239753 -0.75376517 -0.75438723 -0.75113521 -0.75214945 -0.75298478
 -0.75648574 -0.75473404 -0.75458673 -0.75343834 -0.75288419 -0.75099117
 -0.75125453 -0.75113521 -0.75250501 -0.75288419 -0.75250501 -0.75678846
 -0.75113521 -0.75239753 -0.75343834 -0.75288419 -0.75214887 -0.75370135
 -0.75023659 -0.75408832 -0.75113521 -0.75099117 -0.75473404 -0.75350361
 -0.75343834 -0.75340979 -0.75113521 -0.7548147  -0.75214945 -0.75214887
 -0.75113521 -0.75350361 -0.75079461 -0.75214887 -0.75023659 -0.75448728
 -

 -0.75113521 -0.75001649 -0.75076105 -0.75113521]
[-0.75159604 -0.75079461 -0.75159604 -0.75027683 -0.75023659 -0.75027683
 -0.75183046 -0.75027683 -0.7507862  -0.75027683 -0.75183046 -0.75027683
 -0.75027683 -0.75022422 -0.75113521 -0.75023659 -0.7507862  -0.75023659
 -0.75023659 -0.75027683 -0.75022422 -0.75113521 -0.75079461 -0.75022422
 -0.75027683 -0.75094664 -0.75022422 -0.75077176 -0.75096222 -0.75268022
 -0.75023659 -0.75063758 -0.75082212 -0.75027683 -0.75023659 -0.75076105
 -0.75012572 -0.75001649 -0.75059291 -0.75023659 -0.75023659 -0.75058746
 -0.75095132 -0.7506251  -0.75092326 -0.75001649 -0.75023659 -0.75099117
 -0.75125453 -0.75059291 -0.75027683 -0.75288419 -0.75027683 -0.75116403
 -0.75023659 -0.75079461 -0.75027683 -0.75063758 -0.75076105 -0.75113521
 -0.75023659 -0.75094664 -0.75079461 -0.75089698 -0.75023659 -0.75095132
 -0.75086773 -0.75023659 -0.75027683 -0.75125453 -0.75023659 -0.75077176
 -0.75023659 -0.75119378 -0.75023659 -0.75209515 -0.75023659 -0.75076105
 

 -0.74993413 -0.74996456 -0.75004595 -0.7499671 ]
[-0.74993413 -0.74996384 -0.74993413 -0.7500018  -0.75022422 -0.74991122
 -0.75003091 -0.75006724 -0.7500018  -0.74993413 -0.75001649 -0.7500393
 -0.74991122 -0.74993413 -0.75024597 -0.75000679 -0.74996384 -0.74995743
 -0.75000264 -0.74993413 -0.74993413 -0.75008854 -0.74996456 -0.75000668
 -0.75008017 -0.74996384 -0.74993321 -0.75000623 -0.74993478 -0.75000264
 -0.74999118 -0.74996225 -0.74996384 -0.74993413 -0.74993413 -0.74993321
 -0.75012572 -0.74996384 -0.74993372 -0.75015752 -0.74993321 -0.75008017
 -0.74996766 -0.7500379  -0.74991733 -0.75001649 -0.74993413 -0.75000668
 -0.74997055 -0.74996766 -0.75000668 -0.74991623 -0.7500393  -0.74994614
 -0.74996225 -0.75007987 -0.75027683 -0.74991733 -0.75001649 -0.74996766
 -0.75000264 -0.75000264 -0.74993372 -0.74991733 -0.74993655 -0.75013152
 -0.74993413 -0.74993413 -0.74993372 -0.74993478 -0.74995422 -0.75000264
 -0.74993372 -0.74991733 -0.74991122 -0.75008017 -0.74995422 -0.75006746
 -

[-0.74990408 -0.74990269 -0.74993358 -0.74990419 -0.74991018 -0.74991122
 -0.74991122 -0.74990419 -0.74990408 -0.74992627 -0.74990569 -0.7499154
 -0.74990315 -0.74990941 -0.74991434 -0.74990315 -0.74990419 -0.74990495
 -0.74990941 -0.74990495 -0.74991833 -0.7499148  -0.7499113  -0.74991122
 -0.74991381 -0.74990495 -0.74990659 -0.74990315 -0.74990495 -0.74991122
 -0.74990408 -0.74992524 -0.74992635 -0.74990659 -0.74991434 -0.7499148
 -0.74990419 -0.74990408 -0.74990408 -0.74992627 -0.74992635 -0.7499113
 -0.74990258 -0.74991484 -0.74990408 -0.74991309 -0.74990419 -0.74991739
 -0.74993011 -0.74996766 -0.74990659 -0.74990269 -0.74993372 -0.74990408
 -0.74990408 -0.74990408 -0.74990495 -0.74990989 -0.74991122 -0.7499131
 -0.74990495 -0.74992205 -0.74991347 -0.74991122 -0.74990659 -0.74990495
 -0.74990569 -0.74992008 -0.74991309 -0.7499113  -0.74994973 -0.74990408
 -0.74991309 -0.74991484 -0.74990495 -0.74993154 -0.74990469 -0.74991488
 -0.74990269 -0.74990408 -0.7499113  -0.74990408 -0.749

[-0.74990159 -0.74990162 -0.74990411 -0.74990419 -0.74990109 -0.74990162
 -0.74990123 -0.74990123 -0.74990269 -0.74990315 -0.74990569 -0.74990197
 -0.74990199 -0.74990062 -0.74990088 -0.74990197 -0.7499008  -0.74990062
 -0.7499008  -0.74990109 -0.74990057 -0.74990162 -0.74990057 -0.74990659
 -0.74990103 -0.74990088 -0.74990088 -0.74990315 -0.74990123 -0.74990285
 -0.74990045 -0.74990389 -0.74990028 -0.74990062 -0.7499008  -0.74990062
 -0.74990062 -0.74990257 -0.74990315 -0.74990237 -0.74990062 -0.7499008
 -0.7499008  -0.7499008  -0.74990062 -0.74990122 -0.74990062 -0.74990199
 -0.74990461 -0.74990149 -0.74990236 -0.74990197 -0.74990293 -0.74990269
 -0.74990103 -0.74990408 -0.74990123 -0.74990087 -0.74990062 -0.74990162
 -0.74990197 -0.74990062 -0.74990149 -0.74990257 -0.7499037  -0.74990214
 -0.74990087 -0.74990109 -0.74990088 -0.74990162 -0.74990197 -0.74990062
 -0.74990147 -0.74990062 -0.74990062 -0.74990236 -0.7499008  -0.74990147
 -0.74990263 -0.74990102 -0.74990289 -0.74990269 -0.